### This notebook compares the accuracy a logistic regression from scikit-learn vs keras (tensorflow)

In [220]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.datasets import load_digits
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib notebook

In [146]:
data, labels = load_digits(n_class=10, return_X_y=True)
data.shape, labels.shape

((1797, 64), (1797,))

In [230]:
input_size = data.shape[1]
n_classes = len(np.unique(labels))
n_classes

10

In [148]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size=0.7, test_size=0.3, random_state=0)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1257, 64), (1257,), (540, 64), (540,))

### Test using scikit learn

In [238]:
lr = LogisticRegressionCV()
lr.fit(x_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [222]:
pred_y = lr.predict(x_test)
print('Accuracy with scikit-learn LogisticRegressionCV = {:.4f}'.format(lr.score(x_test, y_test)))

Accuracy with scikit-learn LogisticRegressionCV = 0.9648


### Test using keras

In [167]:
def one_hot_encode(arr):
    if arr[0].dtype == np.int:
        return to_categorical(arr)
    
    # Labels must be encoded into numbers 
    classes, ids = np.unique(train_y, return_inverse=True)
    return to_categorical(ids)

In [168]:
y_train_ohe = one_hot_encode(y_train)
y_test_ohe = one_hot_encode(y_test)

In [234]:
model = Sequential()
model.add(Dense(128, input_shape=(input_size,)))
model.add(Activation('sigmoid'))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [235]:
callbacks = [EarlyStopping(monitor='loss', min_delta=1e-10, patience=8, verbose=0, mode='auto')]

In [236]:
model.fit(x_train, y_train_ohe, epochs=100, verbose=1, batch_size=1, callbacks=callbacks)

Epoch 1/100
1257/1257 [==============================] - 2s 1ms/step - loss: 0.7413

In [231]:
preds = model.predict(x_test, verbose=0)
accuracy = np.count_nonzero(np.argmax(preds, axis=1) == y_test) / len(preds)

In [232]:
print('Accuracy with keras = {:.4f}'.format(accuracy))

Accuracy with keras = 0.9759
